# IPython SQL: how to use with PostgreSQL

This notebook introduces [IPython SQL](https://github.com/catherinedevlin/ipython-sql) framework that helps to use [magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) like `%sql` (or `%%sql`) for SQL. We can connect to a database, then issue SQL commands within Jupyter.

We will use connection to PostgreSQL databases. The approach is very simple. 

## Install IPython SQL library

First of all install library with `!pip install` command:

In [ ]:
!pip install ipython-sql

Enable `sql` in Jupyter notebook cells:

In [ ]:
%load_ext sql

## Set credentials

Login and password for access to the database are stored in the environment variables:

In [ ]:
!env | grep POST

In order to connect to PostgreSQL database a connection should be created with parameters like:
- login (or user) `USER`,
- password `PASSWORD`,
- database's host `POSTGRESQL_HOST`,
- database name `DBASE_NAME`.

PostgreSQL has no external IP address, so it is available only from JupyterHub notebooks:

In [ ]:
import os
USER = os.environ['POSTGRESQL_USER']
PASSWORD = os.environ['POSTGRESQL_PASSWORD']
POSTGRESQL_HOST = '10.0.0.28'
DBASE_NAME = 'demo'

Connection data string to be used later:

In [ ]:
CONNECT_DATA = 'postgresql://{}:{}@{}/{}'.format(
    USER,
    PASSWORD,
    POSTGRESQL_HOST,
    DBASE_NAME
)

## Enjoy SQL

Let's look at all databases in PostgreSQL. SQL query can be done after the connection with `%%sql` magic command:

In [ ]:
%%sql $CONNECT_DATA
    SELECT * FROM pg_database

Connect string can be omitted (just use `%sql` NOT the `%%sql`) if connection is done earlier. Now will get all tables in `demo` database:

In [ ]:
%sql SELECT tablename AS table FROM pg_tables WHERE tablename !~ '^(pg_|sql_)'

## Work with result of a query

It is possible to store the result of a query in variable:

In [ ]:
result = %sql SELECT * FROM seats LIMIT 5

In [ ]:
display(result)

In [ ]:
print(result.keys)

In [ ]:
result[0][0]

## More SQL

PostgreSQL database with an address `10.0.0.28` above is read only and is located at stand-alone server. For the educational purposes there is a 'toy' instance of PostgreSQL installed in the container along with Jupyter. 

__NOTE:__ for this section you should start your server with `PostgreSQL environment`.

![Postgres in a box](images/postgres_env.png)

<font color='red'>__VERY IMPORTANT NOTE:__ 'Toy' instance of PostgreSQL installed within 'PostgreSQL environment' was designed only for educational purposes and DOES NOT STORE DATA after you stop your server. You can create or delete tables, write data during session, but next time you start Jupyter server there will be clear database with no data in it.</font>

You can connect to 'toy' database via terminal with `psql` command or with `psycopg2` library:

In [ ]:
import psycopg2

with psycopg2.connect(
    dbname='jovyan'
) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT datname FROM pg_database')
        records = cur.fetchall()
        
print(records)

...or with `%%sql` magic:

In [ ]:
%%sql postgresql:///jovyan
    SELECT * FROM pg_database

This database belong to user of Jupyter and you can do anything you want with it.

### Write a table to database

In [ ]:
%%sql postgresql:///jovyan
CREATE TABLE writers (
    first_name text NOT NULL, 
    last_name text NOT NULL, 
    year_of_death text NOT NULL
);
INSERT INTO writers VALUES ('William', 'Shakespeare', '1616');
INSERT INTO writers VALUES ('Bertold', 'Brecht', '1956')

Check if table `writers` is created:

In [ ]:
%sql SELECT tablename AS table FROM pg_tables WHERE tablename !~ '^(pg_|sql_)'

In [ ]:
%sql SELECT * FROM writers

### Import a demo data into database

You can import [flights demo database](https://postgrespro.com/education/demodb) and work with this data. Here is code how to do it:

In [ ]:
!wget https://edu.postgrespro.com/demo-small-en.zip && unzip -o demo-small-en.zip
!psql -a -f  demo-small-en-20170815.sql
!rm demo-small-en.zip demo-small-en-20170815.sql 

And check if `demo` database apeared:

In [ ]:
%%sql postgresql:///demo
    SELECT * FROM pg_database

In [ ]:
%sql SELECT * FROM seats LIMIT 5